In [121]:
from albumentations import *      # For data augmentation
import cv2                        # For loading images
import os
from shutil import copyfile
from tqdm import tqdm
import splitfolders

In [122]:
#input_folder = "C:\\Users\\kalin\\Desktop\\Hacatons\\sochi\\Full_dataset_2\\"
# splitfolders.ratio(input_folder, 'C:\\Users\\kalin\\Desktop\\Hacatons\\sochi\\Full_dataset_2', ratio = (0.9, 0.1, 0.0), seed=42, group_prefix=None)

In [124]:
transform1 = Compose([
    RandomFog(always_apply=False, p=0.85, fog_coef_lower=0.10, fog_coef_upper=0.10, alpha_coef=0.1),
    Sharpen(always_apply=False, p=0.85, alpha=(0.05, 0.1), lightness=(0.5, 1.0)),
    Spatter(always_apply=False, p=1.0, mean=(0.94, 0.94), std=(50.0, 50.0), gauss_sigma=(2.0, 2.0), intensity=(-0.4, -0.4), cutout_threshold=(0.68, 0.68), mode=['rain']),
    RandomContrast(p=0.85, limit=(-0.40, 0.1))],
    bbox_params=BboxParams(format='yolo'))

c:\Users\kalin\Desktop\Hacatons\sochi\sochi_hack\.venv\Lib\site-packages\albumentations\augmentations\transforms.py:1284: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [125]:
transform2 = Compose([
    GaussianBlur(p = 0.85, blur_limit=(3,3)),
    ToGray(p = 1.0, ),
    CLAHE(p = 0.85, clip_limit=(1, 4), tile_grid_size=(8,8)),
], bbox_params=BboxParams(format='yolo'))

In [126]:
transform3 = Compose([
    ColorJitter(always_apply=False, p=0.75, brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.2, 0.2)),
    Downscale(always_apply=False, p=0.75, scale_min=0.9, scale_max=0.9),
], bbox_params=BboxParams(format='yolo'))

c:\Users\kalin\Desktop\Hacatons\sochi\sochi_hack\.venv\Lib\site-packages\albumentations\augmentations\transforms.py:1692: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


In [127]:
transform4 = Compose([
    Emboss(always_apply=False, p=0.75, alpha=(0.2, 0.35), strength=(1.02, 3.0)),
    Equalize(always_apply=False, p=0.75, mode='cv', by_channels=True),
    ISONoise(always_apply=False, p=0.75, intensity=(0.1, 0.5), color_shift=(0.01, 0.05)),
    RGBShift(always_apply=False, p=0.75, r_shift_limit=(-20, 20), g_shift_limit=(-20, 20), b_shift_limit=(-20, 20))
], bbox_params=BboxParams(format='yolo'))

In [128]:
transform5 = Compose(
    [RandomRain(brightness_coefficient=0.7, drop_width=1, blur_value=2, p=1)], bbox_params=BboxParams(format='yolo'),
)

In [129]:
transform6 = Compose(
    [SafeRotate(always_apply=False, p=1.0, limit=(-19, 19), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None)],
    bbox_params=BboxParams(format='yolo')
)

In [130]:
transform7 = Compose(
    [SafeRotate(always_apply=False, p=1.0, limit=(-19, 19), interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None)], 
     bbox_params=BboxParams(format='yolo')
)

In [131]:
transform8 = Compose(
    [RandomContrast(always_apply=False,
                      p=1.0,
                      limit=(-0.5, 0.01))
    ], bbox_params=BboxParams(format='yolo')
)

In [132]:
transform9 = Compose(
    [PixelDropout(always_apply=False,
                    p=1.0,
                    dropout_prob=0.05,
                    per_channel=0,
                    drop_value=(0, 0, 0),
                    mask_drop_value=None),
     RGBShift(always_apply=False,
                p=1.0,
                r_shift_limit=(-24, 20),
                g_shift_limit=(-24, 20),
                b_shift_limit=(-24, 20))
    ],
    bbox_params=BboxParams(format='yolo')
)

In [133]:
def load_img(path): # Загрузка изображения
    image = cv2.imread(path)
    return image

In [134]:
common_path = 'C:\\Users\\kalin\\Desktop\\Hacatons\\sochi\\sochi_hack\\BackEnd\ml\\augmentations\\train_our'
path_images = os.path.join(common_path, 'images')
path_labels = os.path.join(common_path, 'labels')
for filename in tqdm(os.listdir(path_images)):
    image = load_img(os.path.join(path_images, filename))
    with open(os.path.join(path_labels, f'{filename[:-4]}.txt')) as label_file:
        labels = label_file.read()
        labels = labels[1:]
        labels = list(map(float, labels.split()))
        labels.append('0')
        bboxes = []
        bboxes.append(labels)
    transforms = [transform1, transform2, transform3, transform4, transform5, transform6, transform7, transform8, transform9] # Выбор аугментаций
    if 'NONE' in filename.upper():
        continue
    for i, el in enumerate(transforms):
        try:
            augmented = el(image=image, bboxes=bboxes)
        except Exception:
            break
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
        new_name = f'{filename[:-4]}-{i + 1}'
        cv2.imwrite(os.path.join(path_images, f'{new_name}.jpg'), augmented_image) # запись нового аугментированного файла
        copyfile(os.path.join(path_labels, f'{filename[:-4]}.txt'), (os.path.join(path_labels, f'{new_name}.txt'))) #Идет копирование label

100%|██████████| 2148/2148 [17:41<00:00,  2.02it/s]
